Q1.

In [1]:
import numpy as np

def convolution(data, filter, padding, stride):
  padded = np.pad(data, padding, mode = 'constant', constant_values = 0)
  row = int((data.shape[0]-filter.shape[0]+2*padding)/stride + 1)
  col = int((data.shape[1]-filter.shape[1]+2*padding)/stride + 1)
  filtered = np.zeros((row, col))
  for i in range(row):
    for j in range(col):
      val = padded[stride*i:stride*i+filter.shape[0], stride*j:stride*j+filter.shape[1]]*filter
      filtered[i, j] = val.sum()
  return filtered

In [2]:
a = np.array([[2, 0, 2, 1, 3, 2], 
              [0, 2, 0, 2, 2, 2], 
              [1, 0, 1, 3, 1, 1], 
              [0, 0, 1, 1, 1, 0], 
              [0, 1, 3 ,4 ,1, 0],
              [0, 1, 0, 0, 5, 2]])
b = np.identity(3)

convolution(a, b, 1, 1)

array([[ 4.,  0.,  4.,  3.,  5.,  2.],
       [ 0.,  5.,  3.,  5.,  4.,  5.],
       [ 1.,  1.,  4.,  4.,  3.,  3.],
       [ 1.,  4.,  5.,  3.,  4.,  1.],
       [ 1.,  1.,  3., 10.,  4.,  1.],
       [ 0.,  1.,  1.,  3.,  9.,  3.]])

In [3]:
def maxpool(data, size, padding, stride):
  row = int((data.shape[0]-size+2*padding)/stride + 1)
  col = int((data.shape[1]-size+2*padding)/stride + 1)
  filtered = np.zeros((row, col))
  for i in range(row):
    for j in range(col):
     val = np.max(data[stride*i:stride*i+size, stride*j:stride*j+size])
     filtered[i, j] = val
  return filtered

In [4]:
c = convolution(a, b, 1, 1)
maxpool(c, 2, 0, 2)

array([[ 5.,  5.,  5.],
       [ 4.,  5.,  4.],
       [ 1., 10.,  9.]])

Q2. 

In [5]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(20200813)
if device == "cuda":
  torch.cuda.manual_seed_all(20200813)

In [7]:
learning_rate = 0.0001
training_epochs = 15
batch_size = 100

In [8]:
mnist_train = dsets.MNIST(root='MNIST_data', train=True, download = True,
                      transform=transforms.ToTensor())
mnist_test = dsets.MNIST(root='MNIST_data', train=False, download = True,
                      transform=transforms.ToTensor())

In [9]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [10]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5

    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
    
    self.fc1 = nn.Linear(4*4*128, 625, bias = True)
    nn.init.xavier_uniform_(self.fc1.weight)

    self.layer4 = nn.Sequential(
        self.fc1,
        nn.ReLU(),
        nn.Dropout(p = 1 - self.keep_prob))

    self.fc2 = nn.Linear(625, 10, bias = True)
    nn.init.xavier_uniform_(self.fc2.weight)
    
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out

In [11]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [12]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("[Epoch : {:>4}] cost = {:>.9}".format(epoch+1, avg_cost))

[Epoch :    1] cost = 0.575423241
[Epoch :    2] cost = 0.140181899
[Epoch :    3] cost = 0.0947432071
[Epoch :    4] cost = 0.0740209371
[Epoch :    5] cost = 0.0614111498
[Epoch :    6] cost = 0.0529778749
[Epoch :    7] cost = 0.0466391817
[Epoch :    8] cost = 0.0413130224
[Epoch :    9] cost = 0.0376178138
[Epoch :   10] cost = 0.0337501541
[Epoch :   11] cost = 0.030601291
[Epoch :   12] cost = 0.0291583221
[Epoch :   13] cost = 0.0267761555
[Epoch :   14] cost = 0.0231114924
[Epoch :   15] cost = 0.0215576962


In [13]:
with torch.no_grad():
  model.eval()

  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print("Accuracy:", accuracy.item())

Accuracy: 0.9847999811172485


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [14]:
class CNN2(nn.Module):

  def __init__(self):
    super(CNN2, self).__init__()
    self.keep_prob = 0.5

    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=4, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride = 2, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
    
    self.fc1 = nn.Linear(2*2*128, 625, bias = True)
    nn.init.xavier_uniform_(self.fc1.weight)

    self.layer4 = nn.Sequential(
        self.fc1,
        nn.ReLU(),
        nn.Dropout(p = 1 - self.keep_prob))

    self.fc2 = nn.Linear(625, 10, bias = True)
    nn.init.xavier_uniform_(self.fc2.weight)
    
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out

In [15]:
model = CNN2().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("[Epoch : {:>4}] cost = {:>.9}".format(epoch+1, avg_cost))

[Epoch :    1] cost = 0.793429673
[Epoch :    2] cost = 0.178498879
[Epoch :    3] cost = 0.121410757
[Epoch :    4] cost = 0.0960009024
[Epoch :    5] cost = 0.0794776827
[Epoch :    6] cost = 0.0674418584
[Epoch :    7] cost = 0.0596653707
[Epoch :    8] cost = 0.0538495369
[Epoch :    9] cost = 0.0487324819
[Epoch :   10] cost = 0.0423983559
[Epoch :   11] cost = 0.0395233966
[Epoch :   12] cost = 0.0363375731
[Epoch :   13] cost = 0.0338233076
[Epoch :   14] cost = 0.0321975462
[Epoch :   15] cost = 0.02903153


In [16]:
with torch.no_grad():
  model.eval()

  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print("Accuracy:", accuracy.item())

Accuracy: 0.9896000027656555


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


전 모델에 비하여 cost 증가, accuracy 증가. 하지만 유의미한 차이 없음.